In [1]:
#| default_exp main

In [2]:
#| hide
%load_ext autoreload
%autoreload 2

In [3]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [4]:
#| export
import os, torch, scipy.sparse as sp, joblib, argparse, pickle, numpy as np
from typing import Optional, Dict, Callable, Union

from transformers import DistilBertConfig

from xcai.sdata import SXCDataBlock
from xcai.data import XCDataBlock
from xcai.block import SXCBlock, XCBlock
from xcai.core import get_best_model, load_config
from xcai.transform import AugmentMetaInputIdsTfm

from xclib.utils.sparse import retain_topk

## `Arguement`

In [5]:
#| export
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--build_block', action='store_true')
    parser.add_argument('--use_pretrained', action='store_true')
    
    parser.add_argument('--do_train_inference', action='store_true')
    parser.add_argument('--do_test_inference', action='store_true')
    
    parser.add_argument('--save_train_prediction', action='store_true')
    parser.add_argument('--save_test_prediction', action='store_true')
    parser.add_argument('--save_label_prediction', action='store_true')
    
    parser.add_argument('--save_representation', action='store_true')
    
    parser.add_argument('--use_sxc_sampler', action='store_true')
    parser.add_argument('--only_test', action='store_true')

    parser.add_argument('--pickle_dir', type=str, required=True)
    
    parser.add_argument('--prediction_suffix', type=str, default='')

    parser.add_argument('--exact', action='store_true')
    parser.add_argument('--dataset', type=str)
    
    return parser.parse_args()
    

## Build `block`

In [6]:
#| export
def retain_topk_metadata(block, train_k:int=5, test_k:int=3):
    for meta_name in block.test.dset.meta.keys():
        if train_k is not None and block.train is not None:
            data_meta = retain_topk(block.train.dset.meta[meta_name].data_meta, k=train_k)
            lbl_meta = block.train.dset.meta[meta_name].lbl_meta
            block.train.dset.meta[meta_name].update_meta_matrix(data_meta, lbl_meta)

        if test_k is not None and block.test is not None:
            data_meta = retain_topk(block.test.dset.meta[meta_name].data_meta, k=test_k)
            lbl_meta = block.test.dset.meta[meta_name].lbl_meta
            block.test.dset.meta[meta_name].update_meta_matrix(data_meta, lbl_meta)

def retrain_topk_labels(block, train_k:int=5, test_k:int=3):
    if train_k is not None and block.train is not None:
        block.train.dset.data.data_lbl = retain_topk(block.train.dset.data.data_lbl, k=train_k)
        block.train.dset.data._store_indices()

    if test_k is not None and block.test is not None:
        block.test.dset.data.data_lbl = retain_topk(block.test.dset.data.data_lbl, k=test_k)
        block.test.dset.data._store_indices()
    

In [7]:
#| export
def get_valid_dset(block):
    num_empty_idx = sum(block.dset.data.data_lbl.getnnz(axis=1) == 0)
    return block._getitems(np.where(block.dset.data.data_lbl.getnnz(axis=1) > 0)[0]) if num_empty_idx > 0 else block
    

In [8]:
#| export
def build_block(pkl_file:str, config:Union[str,Dict], use_sxc:Optional[bool]=True, config_key:Optional[str]=None, 
                do_build:Optional[bool]=False, only_test:Optional[bool]=False, remove_empty_datapoints:Optional[bool]=False, 
                train_label_topk:Optional[int]=None, test_label_topk:Optional[int]=None, train_meta_topk:Optional[int]=None, 
                test_meta_topk:Optional[int]=None, meta_name:Optional[str]=None, data_seq_length:Optional[int]=128, 
                lbl_seq_length:Optional[int]=128, exclude_sep:Optional[bool]=False, do_data_meta_aug:Optional[bool]=False,
                do_lbl_meta_aug:Optional[bool]=False, **kwargs):

    if not os.path.exists(pkl_file): do_build = True

    if do_build:
        if isinstance(config, str) and os.path.exists(config): 
            config = load_config(config, config_key)
            if only_test and 'train' in config['path']: del config['path']['train'] 
    
        if use_sxc: 
            block = SXCBlock.from_cfg(config, config_key, padding=True, return_tensors='pt', **kwargs)
        else: 
            block = XCBlock.from_cfg(config, config_key, transform_type='xcs', **kwargs)

            if do_data_meta_aug: block = AugmentMetaInputIdsTfm.apply(block, f'{meta_name}_meta', 'data', data_seq_length, exclude_sep)
            if do_lbl_meta_aug: block = AugmentMetaInputIdsTfm.apply(block, f'{meta_name}_meta', 'lbl', lbl_seq_length, exclude_sep)
            
        joblib.dump(block, pkl_file)
    else:
        block = joblib.load(pkl_file)

        if isinstance(block, SXCDataBlock):
            if 'n_slbl_samples' in kwargs: 
                if block.train is not None: block.train.dset.data.n_slbl_samples = kwargs['n_slbl_samples']
                if block.test is not None: block.test.dset.data.n_slbl_samples = kwargs['n_slbl_samples']
                
            if 'main_oversample' in kwargs: 
                if block.train is not None: block.train.dset.data.main_oversample = kwargs['main_oversample']
                if block.test is not None: block.test.dset.data.main_oversample = kwargs['main_oversample']

            if 'use_main_distribution' in kwargs: 
                if block.train is not None: 
                    block.train.dset.data.use_main_distribution = kwargs['use_main_distribution']
                    if block.train.dset.data.data_lbl_scores is None and kwargs['use_main_distribution']: 
                        block.train.dset.data._store_scores()
                if block.test is not None: 
                    block.test.dset.data.use_main_distribution = kwargs['use_main_distribution']
                    if block.test.dset.data.data_lbl_scores is None and kwargs['use_main_distribution']: 
                        block.test.dset.data._store_scores()

            if 'return_scores' in kwargs:
                if block.train is not None: 
                    block.train.dset.data.return_scores = kwargs['return_scores']
                    if block.train.dset.data.data_lbl_scores is None and kwargs['return_scores']: 
                        block.train.dset.data._store_scores()
                if block.test is not None: 
                    block.test.dset.data.return_scores = kwargs['return_scores']
                    if block.test.dset.data.data_lbl_scores is None and kwargs['return_scores']: 
                        block.test.dset.data._store_scores()

            if block.test is not None:
                for k in block.test.dset.meta:
                    if 'n_sdata_meta_samples' in kwargs: block.test.dset.meta[k].n_sdata_meta_samples = kwargs['n_sdata_meta_samples']
                    if 'n_slbl_meta_samples' in kwargs: block.test.dset.meta[k].n_slbl_meta_samples = kwargs['n_slbl_meta_samples']
                    if 'meta_oversample' in kwargs: block.test.dset.meta[k].meta_oversample = kwargs['meta_oversample']
                    if 'use_meta_distribution' in kwargs: 
                        block.test.dset.meta[k].use_meta_distribution = kwargs['use_meta_distribution']
                        if block.test.dset.meta[k].data_meta_scores is None and kwargs['use_meta_distribution']: 
                            block.test.dset.meta[k]._store_scores()
                    if 'return_scores' in kwargs: 
                        block.test.dset.meta[k].return_scores = kwargs['return_scores']
                        if block.test.dset.meta[k].data_meta_scores is None and kwargs['return_scores']: 
                            block.test.dset.meta[k]._store_scores()

            if block.train is not None:
                for k in block.train.dset.meta:
                    if 'n_sdata_meta_samples' in kwargs: block.train.dset.meta[k].n_sdata_meta_samples = kwargs['n_sdata_meta_samples']
                    if 'n_slbl_meta_samples' in kwargs: block.train.dset.meta[k].n_slbl_meta_samples = kwargs['n_slbl_meta_samples']
                    if 'meta_oversample' in kwargs: block.train.dset.meta[k].meta_oversample = kwargs['meta_oversample']    
                    if 'use_meta_distribution' in kwargs: 
                        block.train.dset.meta[k].use_meta_distribution = kwargs['use_meta_distribution']
                        if block.train.dset.meta[k].data_meta_scores is None and kwargs['use_meta_distribution']: 
                            block.train.dset.meta[k]._store_scores()
                    if 'meta_dropout_remove' in kwargs: block.train.dset.meta[k].meta_dropout_remove = kwargs['meta_dropout_remove']
                    if 'meta_dropout_replace' in kwargs: block.train.dset.meta[k].meta_dropout_replace = kwargs['meta_dropout_replace']
                    if 'return_scores' in kwargs: 
                        block.train.dset.meta[k].return_scores = kwargs['return_scores']
                        if block.train.dset.meta[k].data_meta_scores is None and kwargs['return_scores']: 
                            block.train.dset.meta[k]._store_scores()
                        
    if remove_empty_datapoints: block = type(block)(train=get_valid_dset(block.train), test=get_valid_dset(block.test))

    if train_label_topk is not None or test_label_topk is not None:
        retrain_topk_labels(block, train_k=train_label_topk, test_k=test_label_topk)

    if train_meta_topk is not None or test_meta_topk is not None:
        retain_topk_metadata(block, train_k=train_meta_topk, test_k=test_meta_topk)

    return block
        

## Load `model`

In [9]:
#| export
def load_model(output_dir:str, model_fn:Callable, model_args:Dict, init_fn:Callable, init_args:Optional[Dict]=dict(), 
               do_inference:Optional[bool]=False, use_pretrained:Optional[bool]=False):
    if do_inference:
        os.environ['WANDB_MODE'] = 'disabled'
        if not use_pretrained: model_args['mname'] = f'{output_dir}/{os.path.basename(get_best_model(output_dir))}'

    model = model_fn(**model_args)
    
    if not do_inference or use_pretrained: 
        init_fn(model, **init_args)

    return model
    

## `main` function

In [10]:
#| export
def get_output(pred_idx:torch.Tensor, pred_ptr:torch.Tensor, pred_score:torch.Tensor, n_lbl:int, **kwargs):
    n_data = pred_ptr.shape[0]
    pred_ptr = torch.cat([torch.zeros((1,), dtype=torch.long), pred_ptr.cumsum(dim=0)])
    pred = sp.csr_matrix((pred_score,pred_idx,pred_ptr), shape=(n_data, n_lbl))
    return pred
    

In [11]:
#| export
def main(learn, args, n_lbl:int, eval_dataset=None, train_dataset=None, eval_k:int=None, train_k:int=None, save_teacher:bool=False, 
         save_classifier:bool=False, resume_from_checkpoint:Optional[bool]=None):
    eval_dataset = learn.eval_dataset if eval_dataset is None else eval_dataset
    train_dataset = learn.train_dataset if train_dataset is None else train_dataset
    
    do_infer = args.do_train_inference or args.do_test_inference or args.save_train_prediction \
            or args.save_test_prediction or args.save_representation
    
    if do_infer:
        trn_repr = tst_repr = lbl_repr = trn_pred = tst_pred = None
        prediction_suffix = f'_{args.prediction_suffix}' if len(args.prediction_suffix) else ''
        
        pred_dir = f'{learn.args.output_dir}/predictions'
        os.makedirs(pred_dir, exist_ok=True)

        if args.save_representation:
            trn_repr, lbl_repr = learn.get_data_and_lbl_representation(train_dataset)
            tst_repr = learn._get_data_representation(eval_dataset)

            torch.save(trn_repr, f'{pred_dir}/train_repr{prediction_suffix}.pth')
            torch.save(tst_repr, f'{pred_dir}/test_repr{prediction_suffix}.pth')
            torch.save(lbl_repr, f'{pred_dir}/label_repr{prediction_suffix}.pth')

            if save_teacher:
                teacher = TCH001(DistilBertConfig(), n_data=trn_repr.shape[0], n_lbl=lbl_repr.shape[0])
                teacher.init_embeddings(trn_repr, lbl_repr)
                teacher.freeze_embeddings()
                teacher.save_pretrained(f'{learn.args.output_dir}/teacher')

            if save_classifier:
                classifier = CLS001(DistilBertConfig(), n_train=trn_repr.shape[0], n_test=tst_repr.shape[0], n_lbl=lbl_repr.shape[0])
                classifier.init_representation(trn_repr, tst_repr, lbl_repr)
                classifier.freeze_representation()
                classifier.save_pretrained(f'{learn.args.output_dir}/representation')

        if args.do_test_inference:
            o = learn.predict(eval_dataset)
            print(o.metrics)

            tst_pred = get_output(o.pred_idx, o.pred_ptr, o.pred_score, n_lbl=n_lbl)
            if eval_k is not None: tst_pred = retain_topk(tst_pred, k=eval_k)

            if args.save_test_prediction:
                with open(f'{pred_dir}/test_predictions{prediction_suffix}.pkl', 'wb') as file:
                    pickle.dump(o, file)
                sp.save_npz(f'{pred_dir}/test_predictions{prediction_suffix}.npz', tst_pred)

        if args.do_train_inference:
            o = learn.predict(train_dataset)
            print(o.metrics)

            trn_pred = get_output(o.pred_idx, o.pred_ptr, o.pred_score, n_lbl=n_lbl)
            if train_k is not None: trn_pred = retain_topk(trn_pred, k=train_k)
            
            if args.save_train_prediction:
                with open(f'{pred_dir}/train_predictions{prediction_suffix}.pkl', 'wb') as file:
                    pickle.dump(o, file)
                sp.save_npz(f'{pred_dir}/train_predictions{prediction_suffix}.npz', trn_pred)
                
        return trn_repr, tst_repr, lbl_repr, trn_pred, tst_pred
    else:
        learn.train(resume_from_checkpoint)
        